In [ ]:
import pandas as pd
import os

# Notebookファイルからの相対パスでCSVファイルのパスを指定
csv_file_path = '../data/train.csv'

# ファイルの存在確認
if not os.path.exists(csv_file_path):
    print(f"エラー: ファイルが見つかりません。パスを確認してください: {os.path.abspath(csv_file_path)}")
else:
    try:
        # CSVファイルをDataFrameに読み込む
        # 'time'列 (0番目の列) をインデックスに指定し、同時に日付型としてパースする
        df_train = pd.read_csv(csv_file_path, index_col='time', parse_dates=['time'])
        # または index_col=0, parse_dates=[0] でも同じ結果になります。

        print("DataFrameの読み込みと'time'列のインデックス化・日付型変換が完了しました。")

        # 読み込んだDataFrameの最初の5行を表示して確認
        # print("\nDataFrameの先頭5行:")
        # print(df_train.head())

        # DataFrameのインデックス情報を確認
        # print("\nDataFrameのインデックス情報:")
        # print(df_train.index)

        # DataFrameの基本情報を表示 (time列がインデックスになっていることを確認)
        # print("\nDataFrameの情報:")
        # df_train.info()

    except Exception as e:
        print(f"エラー: CSVファイルの読み込み中に問題が発生しました: {e}")

# この時点で df_train は 'time' 列を DatetimeIndex として持っています。
# この後の欠損値処理などは、この df_train に対して行います。

In [ ]:
pd.set_option('display.max_columns', None)
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
# --- 欠損値補完 ---

# 1. 0で補完するのが適切なカラム
cols_to_fill_zero = [col for col in df_train.columns if 'rain' in col or 'snow' in col]
cols_to_fill_zero.append('generation_solar')

print(f"0で補完するカラム: {cols_to_fill_zero}")
for col in cols_to_fill_zero:
    if col in df_train.columns:
        df_train[col] = df_train[col].fillna(0)

# 2. 残りの連続変数に対して前方補完
# continuous_colsは以前定義した連続変数のリスト
remaining_continuous_cols = [col for col in continuous_cols if col not in cols_to_fill_zero]

print("前方補完（ffill）を実行するカラム（一部抜粋）:", remaining_continuous_cols[:5])
for col in remaining_continuous_cols:
    if col in df_train.columns:
        df_train[col] = df_train[col].fillna(method='ffill')

# 3. それでも先頭に残った欠損を後方補完（bfill）で埋める
# （データの先頭から欠損が続いている場合、ffillでは埋まらないため）
df_train = df_train.fillna(method='bfill')

print("\n欠損値の補完が完了しました。")
# 補完後に欠損がないか確認
print("補完後の欠損値の数:\n", df_train.isnull().sum())

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# このコードを実行する前に、df_trainがロードされている必要があります。
# 例: df_train = pd.read_csv('your_data.csv', index_col='time', parse_dates=True)

# --- ステップ①: 風向をサイン・コサインに変換 ---
print("ステップ①: 風向の処理を開始します...")
cities = ['valencia', 'madrid', 'bilbao', 'barcelona', 'seville']
original_wind_deg_cols = []

for city in cities:
    wind_deg_col = f'{city}_wind_deg'
    original_wind_deg_cols.append(wind_deg_col)

    # 度数をラジアンに変換
    radians = np.deg2rad(df_train[wind_deg_col])

    # 新しいサイン・コサイン特徴量を作成
    df_train[f'{city}_wind_deg_cos'] = np.cos(radians)
    df_train[f'{city}_wind_deg_sin'] = np.sin(radians)

# 元の風向きカラムを削除
df_train = df_train.drop(columns=original_wind_deg_cols)
print(f"元の風向カラム {original_wind_deg_cols} を削除し、サイン・コサインカラムを10個追加しました。")
print("-" * 30)


# --- ステップ②: 連続変数の標準化 ---
print("ステップ②: 連続変数の標準化を開始します...")

# 以前の分類に基づく連続変数のリスト
continuous_cols = [
    'generation_biomass', 'generation_fossil_brown_coal/lignite', 
    'generation_fossil_gas', 'generation_fossil_hard_coal', 'generation_fossil_oil', 
    'generation_hydro_pumped_storage_consumption', 'generation_hydro_run_of_river_and_poundage', 
    'generation_hydro_water_reservoir', 'generation_nuclear', 'generation_other', 
    'generation_other_renewable', 'generation_solar', 'generation_waste', 
    'generation_wind_onshore', 'total_load_actual', 'price_actual'
]
for city in cities:
    continuous_cols.extend([
        f'{city}_temp', f'{city}_temp_min', f'{city}_temp_max', f'{city}_pressure', 
        f'{city}_humidity', f'{city}_wind_speed', f'{city}_rain_1h', f'{city}_rain_3h', 
        f'{city}_snow_3h', f'{city}_clouds_all'
    ])

# ステップ①で追加したサイン・コサインカラムもリストに追加
for city in cities:
    continuous_cols.extend([f'{city}_wind_deg_cos', f'{city}_wind_deg_sin'])

# 欠損値があるとエラーになるため、対象カラムを絞り込み
# （実際には、標準化の前に欠損値処理を行うのが望ましいです）
cols_to_scale = [col for col in continuous_cols if col in df_train.columns and df_train[col].isnull().sum() == 0]
print(f"標準化の対象となるカラム数: {len(cols_to_scale)}")

# 標準化の実行
scaler = StandardScaler()
df_train[cols_to_scale] = scaler.fit_transform(df_train[cols_to_scale])

print("連続変数の標準化が完了しました。")
print("-" * 30)

# 処理後のデータフレームの先頭を表示して確認
print("処理後のデータフレームの先頭5行:")
print(df_train.head())

In [ ]:
import pandas as pd

# このコードを実行する前に、df_trainが前回の処理を終えた状態でロードされている必要があります。

print("処理前のデータフレームの形状:", df_train.shape)
print("処理前のカラム（一部）:", df_train.columns[:10].tolist(), "...")
print("-" * 30)

# --- ステップ①: weather_descriptionカラムを削除 ---
print("ステップ①: weather_descriptionカラムの削除を開始します...")
cities = ['valencia', 'madrid', 'bilbao', 'barcelona', 'seville']
description_cols_to_drop = [f'{city}_weather_description' for city in cities]

# 存在するカラムのみを削除対象とする
existing_cols_to_drop = [col for col in description_cols_to_drop if col in df_train.columns]
if existing_cols_to_drop:
    df_train = df_train.drop(columns=existing_cols_to_drop)
    print(f"削除されたカラム: {existing_cols_to_drop}")
else:
    print("削除対象のdescriptionカラムは見つかりませんでした。")

print("-" * 30)


# --- ステップ②: 残りのカテゴリ変数にOne-Hot Encodingを適用 ---
print("ステップ②: One-Hot Encodingを開始します...")

# One-Hot Encodingの対象となるカラムのリストを作成
categorical_cols_to_encode = []
for city in cities:
    categorical_cols_to_encode.extend([
        f'{city}_weather_id',
        f'{city}_weather_main',
        f'{city}_weather_icon'
    ])

# df_trainに存在するカラムのみを対象とする
existing_categorical_cols = [col for col in categorical_cols_to_encode if col in df_train.columns]

if existing_categorical_cols:
    # weather_idは数値型のため、カテゴリとして正しく扱われるように文字列に変換
    for col in existing_categorical_cols:
        if 'weather_id' in col:
            df_train[col] = df_train[col].astype(str)
    
    print(f"One-Hot Encoding対象のカラム: {existing_categorical_cols}")
    
    # One-Hot Encodingを実行
    # prefixを指定して、どの元のカラムから来た特徴量か分かりやすくします。
    # 元のカテゴリカルカラムは自動的に削除されます。
    df_train = pd.get_dummies(df_train, columns=existing_categorical_cols)
    
    print("One-Hot Encodingが完了しました。")
else:
    print("One-Hot Encoding対象のカラムは見つかりませんでした。")

print("-" * 30)
print("全ての処理が完了しました。")
print("最終的なデータフレームの形状:", df_train.shape)
print("処理後のカラム（一部）:", df_train.columns[:10].tolist(), "...")
print("\n処理後のデータフレームの先頭5行:")
print(df_train.head())

In [ ]:
df_train["price_actual"].describe()

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# df_train が前処理済みの状態で存在すると仮定します。
# 例：df_train = pd.read_csv(...) などでロードされ、
# 欠損値補完、標準化、One-Hotエンコーディングが完了している状態。

In [ ]:
def create_sequences(input_data, target_column, timesteps):
    """
    2Dの時系列データフレームから、LSTM用の3Dシーケンスデータを作成する関数
    """
    X_list, y_list = [], []
    
    # 説明変数と目的変数に分離
    # One-Hotエンコーディングにより目的変数の位置が変わっている可能性があるため、名前で指定
    y = input_data[target_column].values
    X_data = input_data.drop(columns=target_column).values
    
    for i in range(len(X_data) - timesteps):
        X_list.append(X_data[i:i+timesteps])
        y_list.append(y[i+timesteps])
        
    return np.array(X_list), np.array(y_list)

# ハイパーパラメータ
TIMESTEPS = 24
TARGET_COLUMN = 'price_actual' # 目的変数のカラム名

# ★★★ 修正箇所：ここから ★★★
# bool型をint型に変換して、データ型を数値に統一します
bool_cols = df_train.select_dtypes(include=['bool']).columns
df_train[bool_cols] = df_train[bool_cols].astype(int)
print("bool型をint型に変換しました。")
# ★★★ 修正箇所：ここまで ★★★

print("シーケンスデータを作成中...")
X_seq, y_seq = create_sequences(df_train, TARGET_COLUMN, TIMESTEPS)

print(f"X_seq shape: {X_seq.shape}") # -> (サンプル数, 24, 特徴量数)
print(f"y_seq shape: {y_seq.shape}") # -> (サンプル数,)

In [ ]:
# 訓練データと検証データの分割（例: 80%を訓練、20%を検証）
train_size = int(len(X_seq) * 0.8)
X_train, X_val = X_seq[:train_size], X_seq[train_size:]
y_train, y_val = y_seq[:train_size], y_seq[train_size:]

# PyTorchのテンソルに変換
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(1)

# DatasetとDataLoaderの作成
BATCH_SIZE = 64 # バッチサイズも後で調整するハイパーパラメータ

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("DataLoaderの準備が完了しました。")

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout_prob):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout_prob if num_layers > 1 else 0 # 1層の場合はdropoutは適用されない
        )
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # LSTM層からの出力を取得
        # h_n, c_n (最後の隠れ状態とセル状態) は今回は使用しない
        lstm_out, _ = self.lstm(x)
        
        # 最後のタイムステップの出力のみを選択
        last_hidden_state = lstm_out[:, -1, :]
        
        # ドロップアウト層
        out = self.dropout(last_hidden_state)
        
        # 全結合層（出力層）
        out = self.fc(out)
        return out

In [ ]:
# ハイパーパラメータ（Optunaで探索する候補）
INPUT_SIZE = X_train.shape[2] # 特徴量の数
HIDDEN_SIZE = 100
NUM_LAYERS = 2
DROPOUT_PROB = 0.2
LEARNING_RATE = 0.001
NUM_EPOCHS = 20 # まずは少ないエポック数で試す

# デバイスの設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用デバイス: {device}")

# モデル、損失関数、オプティマイザのインスタンス化
model = LSTMModel(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, DROPOUT_PROB).to(device)
loss_fn = nn.MSELoss() # 平均二乗誤差
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

print("モデルの学習を開始します...")
for epoch in range(NUM_EPOCHS):
    # --- 訓練モード ---
    model.train()
    total_train_loss = 0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        predictions = model(inputs)
        loss = loss_fn(predictions, labels)
        loss.backward()
        optimizer.step()
        
        total_train_loss += loss.item()
    
    avg_train_loss = total_train_loss / len(train_loader)

    # --- 検証モード ---
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            predictions = model(inputs)
            loss = loss_fn(predictions, labels)
            total_val_loss += loss.item()
            
    avg_val_loss = total_val_loss / len(val_loader)
    
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] | Train Loss: {avg_train_loss:.4f} | Validation Loss: {avg_val_loss:.4f}")

In [ ]:
import pandas as pd
import os

# Notebookファイルからの相対パスでCSVファイルのパスを指定
csv_file_path = '../data/test.csv'

# ファイルの存在確認
if not os.path.exists(csv_file_path):
    print(f"エラー: ファイルが見つかりません。パスを確認してください: {os.path.abspath(csv_file_path)}")
else:
    try:
        # CSVファイルをDataFrameに読み込む
        # 'time'列 (0番目の列) をインデックスに指定し、同時に日付型としてパースする
        df_test = pd.read_csv(csv_file_path, index_col='time', parse_dates=['time'])
        # または index_col=0, parse_dates=[0] でも同じ結果になります。

        print("DataFrameの読み込みと'time'列のインデックス化・日付型変換が完了しました。")

        # 読み込んだDataFrameの最初の5行を表示して確認
        # print("\nDataFrameの先頭5行:")
        # print(df_test.head())

        # DataFrameのインデックス情報を確認
        # print("\nDataFrameのインデックス情報:")
        # print(df_test.index)

        # DataFrameの基本情報を表示 (time列がインデックスになっていることを確認)
        # print("\nDataFrameの情報:")
        # df_test.info()

    except Exception as e:
        print(f"エラー: CSVファイルの読み込み中に問題が発生しました: {e}")

# この時点で df_test は 'time' 列を DatetimeIndex として持っています。
# この後の欠損値処理などは、この df_test に対して行います。

In [ ]:
df_test.info()

In [ ]:
import pandas as pd
import numpy as np

# このコードは、df_trainの前処理が完了し、
# scaler オブジェクトと、最終的な df_train が存在することを前提とします。
# df_test には目的変数 'price_actual' が含まれていないと仮定します。

print("テストデータの前処理を開始します...")
print(f"処理前のdf_testの形状: {df_test.shape}")

# --- ステップ1: 欠損値補完 ---
# df_trainと同じ戦略で補完
print("1. 欠損値を補完中...")
cols_to_fill_zero = [col for col in df_test.columns if 'rain' in col or 'snow' in col or 'solar' in col]
df_test[cols_to_fill_zero] = df_test[cols_to_fill_zero].fillna(0)
df_test = df_test.fillna(method='ffill').fillna(method='bfill')

# --- ステップ2: 風向の処理と不要カラムの削除 ---
print("2. 風向をサイン・コサインに変換し、不要なカラムを削除中...")
cities = ['valencia', 'madrid', 'bilbao', 'barcelona', 'seville']

# 風向の処理
original_wind_deg_cols = [f'{city}_wind_deg' for city in cities]
for col in original_wind_deg_cols:
    if col in df_test.columns:
        radians = np.deg2rad(df_test[col])
        df_test[f'{col}_cos'] = np.cos(radians)
        df_test[f'{col}_sin'] = np.sin(radians)
df_test = df_test.drop(columns=original_wind_deg_cols)

# weather_descriptionの削除
description_cols_to_drop = [f'{city}_weather_description' for city in cities]
df_test = df_test.drop(columns=description_cols_to_drop, errors='ignore')

# --- ステップ3: 標準化 (訓練時のScalerを再利用) ---
# df_trainの前処理で定義した continuous_cols リストが必要
print("3. 訓練時のScalerを再利用して標準化中...")

# df_trainの処理で使用したcontinuous_colsのリストから、df_testに存在するカラムのみを対象にする
# 'price_actual'はdf_testに存在しないため、自動的に除外される
cols_to_scale_test = [col for col in continuous_cols if col in df_test.columns]

# ★重要★ 訓練データで学習したscalerを使い、transformのみ適用
df_test[cols_to_scale_test] = scaler.transform(df_test[cols_to_scale_test])


# --- ステップ4: One-Hot Encoding とカラムの整合 ---
print("4. One-Hot Encodingとカラムの整合性を確保中...")

# OHE対象カラムのリストを作成
categorical_cols_to_encode = []
for city in cities:
    categorical_cols_to_encode.extend([f'{city}_weather_id', f'{city}_weather_main', f'{city}_weather_icon'])

# 存在するカラムのみを対象とし、カテゴリとして扱うために文字列に変換
existing_categorical_cols = [col for col in categorical_cols_to_encode if col in df_test.columns]
for col in existing_categorical_cols:
    df_test[col] = df_test[col].astype(str)

# One-Hot Encodingを実行
df_test = pd.get_dummies(df_test, columns=existing_categorical_cols)

# ★重要★ df_trainのカラム構成にdf_testを完全に合わせる
train_columns = df_train.drop(columns=[TARGET_COLUMN]).columns
df_test = df_test.reindex(columns=train_columns, fill_value=0)

# bool型をint型に変換
bool_cols = df_test.select_dtypes(include=['bool']).columns
df_test[bool_cols] = df_test[bool_cols].astype(int)


print("\n前処理が完了しました。")
print(f"処理後のdf_testの形状: {df_test.shape}")
print(f"比較（df_trainの形状）: ({df_train.shape[0]}, {df_train.shape[1]-1})") # 目的変数を引いた数

# カラムが一致しているか最終確認
if len(train_columns) == len(df_test.columns) and all(train_columns == df_test.columns):
    print("OK: df_trainとdf_testのカラム構成が一致しました。")
else:
    print("NG: df_trainとdf_testのカラム構成が一致していません。見直しが必要です。")